<a href="https://colab.research.google.com/github/ryang132/MidTerm-Project/blob/main/RobertYang_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the Dataset provided and upload to the drive


In [ ]:
#unzip the dataset
!unzip -q tiny_Data.zip -d Data

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import cv2
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataGenerator(Sequence):
    def __init__(self, directory, batch_size, image_size, augment=False):
        self.directory = directory
        self.batch_size = batch_size
        self.image_size = image_size
        self.augment = augment
        self.classes = sorted(os.listdir(directory))
        self.num_classes = len(self.classes)
        self.class_indices = dict(zip(self.classes, range(self.num_classes)))
        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(directory, class_name)
            for filename in os.listdir('/content/Data/Data_Small/train'):
                self.samples.append((os.path.join(class_dir, filename), class_name))
        self.num_samples = len(self.samples)

    def __len__(self):
        return int(np.ceil(self.num_samples / float(self.batch_size)))

    def __getitem__(self, index):
        batch_samples = self.samples[index*self.batch_size:(index+1)*self.batch_size]
        X = np.zeros((len(batch_samples), *self.image_size, 3), dtype=np.float32)
        y = np.zeros((len(batch_samples), len(self.classes)), dtype=np.float32)
        for i, (filename, class_name) in enumerate(batch_samples):
            try:
                img = cv2.imread(filename)
                if img is None:
                    continue  # Skip over images that cannot be loaded
                img = cv2.resize(img, self.image_size)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = np.array(img) / 255.0
                X[i] = img
                y[i, self.class_indices[class_name]] = 1
            except Exception as e:
                print(f"Error loading image {filename}: {e}")
        return X, y

    def _augment_image(self, img):
        # Create an instance of the image data generator
        datagen = ImageDataGenerator(rotation_range=20,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
        # Add a batch dimension to the image and apply augmentation
        img = np.expand_dims(img, axis=0)
        img = datagen.flow(img, batch_size=1)[0][0]
        # Remove the batch dimension from the augmented image
        img = np.squeeze(img, axis=0)
        return img


# Define data directories
train_dir = '/content/Data/Data_Small/train/DME'
val_dir = '/content/Data/Data_Small/train/DRUSEN'

# Define model parameters
img_width, img_height = 256, 256
batch_size = 32
epochs = 10

# Define data generators
train_generator = DataGenerator(train_dir, batch_size, image_size=(img_width, img_height), augment=True)
val_generator = DataGenerator(val_dir, batch_size, image_size=(img_width, img_height))

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])



# Train the model
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)




Epoch 1/10
95/95 [==============================] - 636s 7s/step - loss: 0.6659 - accuracy: 1.0000 - val_loss: 0.6426 - val_accuracy: 1.0000
Epoch 2/10
95/95 [==============================] - 628s 7s/step - loss: 0.6209 - accuracy: 1.0000 - val_loss: 0.5989 - val_accuracy: 1.0000
Epoch 3/10
95/95 [==============================] - 641s 7s/step - loss: 0.5782 - accuracy: 1.0000 - val_loss: 0.5574 - val_accuracy: 1.0000
Epoch 4/10
95/95 [==============================] - 629s 7s/step - loss: 0.5378 - accuracy: 1.0000 - val_loss: 0.5181 - val_accuracy: 1.0000
Epoch 5/10
95/95 [==============================] - 634s 7s/step - loss: 0.4996 - accuracy: 1.0000 - val_loss: 0.4810 - val_accuracy: 1.0000
Epoch 6/10
95/95 [==============================] - 628s 7s/step - loss: 0.4636 - accuracy: 1.0000 - val_loss: 0.4460 - val_accuracy: 1.0000
Epoch 7/10
95/95 [==============================] - 626s 7s/step - loss: 0.4295 - accuracy: 1.0000 - val_loss: 0.4130 - val_accuracy: 1.0000
Epoch 8/10
95